In [2]:
import numpy as np
from function import crcbgenqcsig, normsig4psd, statgaussnoisegen, innerprod_psd


def glrtqcsig(time_vec,data_vec, fs,psd_vec,qcCoefs):

    "calculate the GLRT for a quadratic chirp signal with unknown amplitude"
    'psd_vec: for positive DFT frequencies'
    
    # Generate unit norm template
    sig_vec = crcbgenqcsig(time_vec, 1, qcCoefs)
    templateVec, _ = normsig4psd(sig_vec, fs, psd_vec, 1)

    # Calculate inner product of data with unit norm template
    llr = innerprod_psd(data_vec, templateVec, fs, psd_vec)

    return llr**2

In [3]:
#example usage of the GLRT function

# Parameters for data realization
nSamples = 2048
sampFreq = 1024
timeVec = np.arange(nSamples) / sampFreq

# Supply PSD values
def noisePSD(f):
    """Noise power spectral density function"""
    return ((f >= 100) & (f <= 300)) * (f - 100) * (300 - f) / 10000 + 1

dataLen = nSamples / sampFreq
kNyq = nSamples // 2 + 1
posFreq = np.arange(kNyq) / dataLen
psdPosFreq = noisePSD(posFreq)

# Generate data realization
# Parameters of the signal to be injected
a1 = 9.5
a2 = 2.8
a3 = 3.2
A = 200  # SNR

# Generate signal with arbitrary amplitude
sig4data = crcbgenqcsig(timeVec, 1, [a1, a2, a3])
# Normalize signal to SNR = A in noise with specified PSD
sig4data, _ = normsig4psd(sig4data, sampFreq, psdPosFreq, A)
print('SNR of the signal:', A)

# Generate noise realization
noiseVec = statgaussnoisegen(nSamples, np.column_stack([posFreq, psdPosFreq]), 100, sampFreq)

# Data realization = noise + signal
dataVec = noiseVec + sig4data

# Compute GLRT
glrt_value=glrtqcsig(timeVec,dataVec,sampFreq,psdPosFreq,[a1, a2, a3])

print(f"GLRT value: {glrt_value:.4f}")


SNR of the signal: 200
GLRT value: 39797.1907
